In [1]:
import numpy

In [2]:
class GaussMatrix:
    """Расширенная матрица. Нулевой столбец - для свободных членов, положительные - для матрицы коэффициентов (индексы начинаются с 1), а в отрицательных автоматически появится единичная матрица a[i, -i] = 1"""

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1] + self.n
            return self.m[i, j]
        else:
            return self.m[index - 1]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1] + self.n
            self.m[i, j] = value
        else:
            self.m[index - 1] = value

    def __init__(self, nrows):
        self.n = nrows
        self.m = numpy.zeros((nrows, 2 * nrows + 1))
        
        for i in range(1, nrows + 1):
            self[i, -i] = 1


# Алгоритм начинается здесь
    
    def forwardGauss(self):
        """Прямой ход метода Гаусса"""
        for i in range(1, self.n + 1):

            ii = self[i, i]
            self[i] /= ii

            for i_ in range(i + 1, self.n + 1):
                i_i = self[i_, i]
                self[i_] -= self[i] * i_i
                
    def backwardGauss(self):
        """Обратный ход метода Гаусса"""
        for i in range(self.n, 0, -1):
            for i_ in range(i-1, 0, -1):
                i_i = self[i_, i]
                self[i_] -= self[i] * i_i

    
    def solveGauss(self):
        """Решает систему методом Гаусса, возвращает список найденных иксов. Предполагается, что есть диагональное преобладание."""
        self.forwardGauss()
        self.backwardGauss()
        return [self[i, 0] for i in range(1, self.n+1)]


In [3]:
class Matrix:
    """Чтобы использовать индексы, начиная с 1, а в нулевом столбце хранить свободные члены"""
    def __getitem__(self, index):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1]
            return self.m[i, j]
        else:
            return self.m[index - 1]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i = index[0] - 1
            j = index[1]
            self.m[i, j] = value
        else:
            self.m[index - 1] = value

    def __init__(self, n):
        self.n = n
        self.m = numpy.zeros((n, n + 1))



    def norm(self):
        """1 норма матрицы - максимальная сумма модулей в строке"""
        sumofmodulesarray = []

        for i in range(1, self.n + 1):
            sumofmodules = 0

            for j in range(1, self.n + 1):
                sumofmodules += abs(self[i, j])
            sumofmodulesarray.append(sumofmodules)

        return max(sumofmodulesarray)

    def dot(self, x):
        """Умножает матрицу на вектор с учётом съехавших индексов"""
        x_ = [0] + x
        return self.m.dot(x_)

In [4]:
class LinearSystem:
    """Сохраняет исходную матрицу в атрибуте source : Matrix, реализует метод Гаусса, Зейделя и умеет считать число обусловленности."""
    def __getitem__(self, index):
        return self.source.__getitem__(index)
    def __setitem__(self, index, value):
        self.source.__setitem__(index, value)

    def __init__(self, n):
        self.n = n
        self.source = Matrix(n)

    def solveGauss(self):
        """Решает систему методом Гаусса, возвращает список найденных иксов. Предполагается, что есть диагональное преобладание."""
        self.g = GaussMatrix(self.n)

        for i in range(1, self.n + 1):
            for j in range(0, self.n + 1):
                self.g[i, j] = self[i, j]
        
        ans = self.g.solveGauss()

        # Надо сохранить обратную матрицу, которая получилась как побочный продукт при решении методом Гаусса, чтобы потом считать число обусловленности
        self.inverse = Matrix(self.n)
        for i in range(self.n+1):
            for j in range(self.n+1):
                self.inverse[i, j] = self.g[i, -j]
        return ans

    def conditionNumber(self):
        """Число обусловленности по первой норме. Считает обратную матрицу по методу Гаусса, если она уже не посчитана."""
        if not hasattr(self, "inverse"):
            self.solveGauss()
        return self.source.norm() * self.inverse.norm()







    def solveSeidel(self, max_residual):
        """Решает систему методом Зейделя, возвращает список найденных иксов. Предполагается, что метод сходится."""
        ld = GaussMatrix(self.n)
        for i in range(1, self.n + 1):
            for j in range(1, i + 1):
                ld[i, j] = self.source[i, j]
        ld.forwardGauss()

        ld_inv = numpy.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(self.n):
                ld_inv[i, j] = ld.m[i, self.n - j - 1] # эквив ld[i', -j'], где i' = i + 1, j' = j + 1

        A = numpy.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(self.n):
                A[i, j] = self.source[i + 1, j + 1]

        u = numpy.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(i + 1, self.n):
                u[i, j] = A[i, j]

        ld_inv_times_u = numpy.matmul(ld_inv, u)
        b = numpy.array([self.source[i, 0] for i in range(1, self.n+1)])
        ld_inv_times_b = ld_inv.dot(b)

        x = numpy.zeros(self.n)
        while numpy.max(abs(A.dot(x) - b)) > max_residual:
            x = - ld_inv_times_u.dot(x) + ld_inv_times_b

        return x.tolist()


Содержательный код закончился, дальше я его запускаю

In [5]:
def scaryMatrix(nrows):
    """Обобщение системы из условия на случай матриц произвольного размера.
        @returns LinearSystem"""
    m = LinearSystem(nrows)

    for i in range(2, nrows):
        m[i, i] = 10
        m[i, i-1] = m[i, i+1] = 1
    m[1, 1] = m[nrows, nrows] = 10
    m[1, 2] = 1

    for i in range(1, nrows+1):
        m[nrows, i] = 1
        
    for i in range(1, nrows+1):
        m[i, 0] = i
    return m

s = scaryMatrix(100)

In [6]:
ansGauss = s.solveGauss()
ansGauss

[0.08333333333333334,
 0.16666666666666666,
 0.24999999999999997,
 0.3333333333333333,
 0.4166666666666667,
 0.5,
 0.5833333333333334,
 0.6666666666666666,
 0.75,
 0.8333333333333334,
 0.9166666666666667,
 1.0000000000000002,
 1.0833333333333335,
 1.1666666666666667,
 1.25,
 1.3333333333333335,
 1.4166666666666667,
 1.5000000000000002,
 1.5833333333333335,
 1.6666666666666667,
 1.7500000000000002,
 1.8333333333333337,
 1.916666666666667,
 1.9999999999999998,
 2.0833333333333335,
 2.1666666666666665,
 2.2500000000000004,
 2.3333333333333335,
 2.4166666666666665,
 2.5,
 2.583333333333333,
 2.666666666666667,
 2.75,
 2.833333333333333,
 2.916666666666667,
 3.0,
 3.083333333333334,
 3.1666666666666665,
 3.25,
 3.3333333333333335,
 3.416666666666666,
 3.5000000000000004,
 3.583333333333333,
 3.6666666666666674,
 3.7499999999999996,
 3.8333333333333335,
 3.916666666666667,
 4.0,
 4.083333333333333,
 4.166666666666667,
 4.25,
 4.333333333333334,
 4.416666666666666,
 4.5,
 4.583333333333334,
 

In [7]:
# Проверяем, что решение верно

s.source.dot(ansGauss) # - [i for i in range(1, 101)]

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])

In [8]:
# Число обусловленности
s.conditionNumber()

1020.127651091681

In [9]:
ansSeidel = s.solveSeidel(1e-8)
ansSeidel

[0.08333333333292736,
 0.16666666666626886,
 0.2499999999995725,
 0.33333333333287957,
 0.4166666666661863,
 0.4999999999994933,
 0.5833333333327999,
 0.6666666666661067,
 0.7499999999994134,
 0.83333333333272,
 0.9166666666660271,
 0.9999999999993339,
 1.0833333333326405,
 1.1666666666659473,
 1.249999999999254,
 1.333333333332561,
 1.4166666666658678,
 1.4999999999991742,
 1.5833333333324813,
 1.6666666666657879,
 1.7499999999990945,
 1.8333333333324013,
 1.9166666666657084,
 1.9999999999990155,
 2.083333333332322,
 2.1666666666656287,
 2.2499999999989355,
 2.333333333332243,
 2.416666666665549,
 2.4999999999988556,
 2.583333333332162,
 2.6666666666654693,
 2.749999999998776,
 2.8333333333320834,
 2.9166666666653898,
 2.999999999998696,
 3.0833333333320034,
 3.1666666666653103,
 3.2499999999986176,
 3.3333333333319235,
 3.4166666666652312,
 3.499999999998537,
 3.5833333333318436,
 3.6666666666651517,
 3.7499999999984563,
 3.833333333331765,
 3.9166666666650705,
 3.999999999998379,
 4

In [13]:
# Проверяем, что решение верно

s.source.dot(ansSeidel) # - [i for i in range(1, 101)]

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])